In [14]:
from harvesters.core import Harvester
import numpy as np
from harvesters.util.pfnc import mono_location_formats, rgb_formats, bgr_formats, rgba_formats, bgra_formats

from PIL import Image
import numpy as np

import cv2
import time


In [15]:
h = Harvester() 

In [16]:
h.add_file('C:\Program Files\Baumer GAPI SDK (64 Bit)\Tools\CameraExplorer\mvGenTLProducer.cti');

In [17]:
h.files

['C:\\Program Files\\Baumer GAPI SDK (64 Bit)\\Tools\\CameraExplorer\\mvGenTLProducer.cti']

In [18]:
h.update()

In [19]:
for i in h.device_info_list:
    print(i)

In [20]:
ia = h.create_image_acquirer(1)

IndexError: list index out of range

In [ ]:
ia.remote_device.node_map.Width.value = 4000
ia.remote_device.node_map.Height.value = 3000
#ia.remote_device.node_map.PixelFormat.value = 'Mono8'

In [ ]:
ia.start_acquisition()

In [ ]:
while True:
    with ia.fetch_buffer() as buffer:
     # Work with the Buffer object. It consists of everything you need.
        component = buffer.payload.components[0]

        width = component.width
        height = component.height
        data_format = component.data_format

        if data_format in mono_location_formats: 
            content = component.data.reshape(height, width) 
        else:
            if data_format in rgb_formats or data_format in rgba_formats or data_format in bgr_formats or data_format in bgra_formats: #
                content = component.data.reshape( height, width, int(component.num_components_per_pixel)) # Set of R, G, B, and Alpha ) #
                if data_format in bgr_formats: # Swap every R and B:
                    content = content[:, :, ::-1] 
                    
        scale_percent = 30
        width = int(content.shape[1] * scale_percent / 100)
        height = int(content.shape[0] * scale_percent / 100)
        dim = (width, height)

        cv2.imshow("Image", cv2.resize(content, dim, interpolation = cv2.INTER_AREA))
        if cv2.waitKey(5)== ord('q'):
            cv2.destroyAllWindows()
            break
                                              

In [ ]:
im = Image.fromarray(content , 'RGB')
im.save('test1.bmp')

In [ ]:
ia.stop_acquisition()
ia.destroy()

In [ ]:
h.reset()